In [ ]:
# TODO: FOR MYSELF IN THE MORNING
#   - Currently the count checks if there's 4 occurrences after grouping the data which assumes 15-minute records, change so it's calculated depending on the exact minute record
#   - If samples aren't recorded in equal intervals, maybe somehow filter data depending on the interval, apply the transform and then concat into final DF

In [241]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from functools import reduce

import warnings

warnings.filterwarnings('ignore')

In [2]:
DATA_PATH = '../data/'

In [3]:
# Save names of subdirectories within ./data/ directory into a list
data_dirs = [name for name in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, name))]
print(data_dirs)

['SE', 'DE', 'HU', 'IT', 'SP', 'NE', 'PO', 'UK', 'DK']


In [4]:
green_energy_sources = {
    'B01': 'Biomass',
    'B09': 'Geothermal',
    'B10': 'Hydro Pumped Storage',
    'B11': 'Hydro Run-of-river and poundage',
    'B12': 'Hydro Water Reservoir',
    'B13': 'Marine',
    'B15': 'Other renewable',
    'B16': 'Solar',
    'B17': 'Waste',
    'B18': 'Wind Offshore',
    'B19': 'Wind Onshore'
}

In [5]:
energy_categories = {
    'A03': 'Mixed',
    'A04': 'Generation',
    'A05': 'Load',
    'B01': 'Biomass',
    'B02': 'Fossil Brown coal/Lignite',
    'B03': 'Fossil Coal-derived gas',
    'B04': 'Fossil Gas',
    'B05': 'Fossil Hard coal',
    'B06': 'Fossil Oil',
    'B07': 'Fossil Oil shale',
    'B08': 'Fossil Peat',
    'B09': 'Geothermal',
    'B10': 'Hydro Pumped Storage',
    'B11': 'Hydro Run-of-river and poundage',
    'B12': 'Hydro Water Reservoir',
    'B13': 'Marine',
    'B14': 'Nuclear',
    'B15': 'Other renewable',
    'B16': 'Solar',
    'B17': 'Waste',
    'B18': 'Wind Offshore',
    'B19': 'Wind Onshore',
    'B20': 'Other',
    'B21': 'AC Link',
    'B22': 'DC Link',
    'B23': 'Substation',
    'B24': 'Transformer'
}

In [7]:
list(green_energy_sources.keys())

['B01', 'B09', 'B10', 'B11', 'B12', 'B13', 'B15', 'B16', 'B17', 'B18', 'B19']

In [187]:
df = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/gen_HU_B01.csv')
df2 = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/gen_HU_B11.csv')
df3 = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/gen_HU_B14.csv')

In [188]:
df['StartTime'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['StartTime'] = pd.to_datetime(df['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

df2['StartTime'] = df2['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df2['StartTime'] = pd.to_datetime(df2['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

df3['StartTime'] = df3['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df3['StartTime'] = pd.to_datetime(df3['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

In [173]:
df2.head()

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2021-12-31 23:45:00,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B11,11
1,2022-01-01 00:00:00,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B11,11
2,2022-01-01 00:15:00,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B11,11
3,2022-01-01 00:30:00,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B11,11
4,2022-01-01 00:45:00,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B11,11


In [189]:
dfs = [df, df2, df3]

In [190]:
print(df.head(1))
print(df2.head(1))
print(df3.head(1))

            StartTime                  EndTime            AreaID UnitName  \
0 2021-12-31 23:45:00  2022-01-01T00:00+00:00Z  10YHU-MAVIR----U      MAW   

  PsrType  quantity  
0     B01       132  
            StartTime                  EndTime            AreaID UnitName  \
0 2021-12-31 23:45:00  2022-01-01T00:00+00:00Z  10YHU-MAVIR----U      MAW   

  PsrType  quantity  
0     B11        11  
            StartTime                  EndTime            AreaID UnitName  \
0 2021-12-31 23:45:00  2022-01-01T00:00+00:00Z  10YHU-MAVIR----U      MAW   

  PsrType  quantity  
0     B14      1950  


In [182]:
def custom_merge(df_left, df_right):
    merged_df = pd.merge(df_left, df_right, on=['StartTime'], how='outer', suffixes=('', '_other'))
    # Sum up the 'quantity' values from both dataframes
    merged_df['quantity'] = merged_df['quantity'] + merged_df['quantity_other']
    # Drop the redundant 'quantity_other' column
    merged_df = merged_df.drop('quantity_other', axis=1)
    return merged_df

In [191]:
df_merged = reduce(custom_merge, dfs)

In [192]:
df_merged.head()

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,EndTime_other,AreaID_other,UnitName_other,PsrType_other,EndTime_other,AreaID_other,UnitName_other,PsrType_other
0,2021-12-31 23:45:00,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B01,2093,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B11,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B14
1,2022-01-01 00:00:00,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B01,2091,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B11,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B14
2,2022-01-01 00:15:00,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B01,2091,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B11,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B14
3,2022-01-01 00:30:00,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B01,2090,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B11,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B14
4,2022-01-01 00:45:00,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B01,2090,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B11,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B14


In [178]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['StartTime'], how='outer', suffixes=('', '_other')), dfs)
df_merged['quantity'] = df_merged['quantity'] + df_merged['quantity_other']
df_merged.head()

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,EndTime_other,AreaID_other,UnitName_other,PsrType_other,quantity_other
0,2021-12-31 23:45:00,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B01,143,2022-01-01T00:00+00:00Z,10YHU-MAVIR----U,MAW,B11,11
1,2022-01-01 00:00:00,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B01,142,2022-01-01T00:15+00:00Z,10YHU-MAVIR----U,MAW,B11,11
2,2022-01-01 00:15:00,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B01,142,2022-01-01T00:30+00:00Z,10YHU-MAVIR----U,MAW,B11,11
3,2022-01-01 00:30:00,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B01,142,2022-01-01T00:45+00:00Z,10YHU-MAVIR----U,MAW,B11,11
4,2022-01-01 00:45:00,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B01,141,2022-01-01T01:00+00:00Z,10YHU-MAVIR----U,MAW,B11,11


In [47]:
df.loc[df['quantity'] <1000].shape

(96, 6)

In [382]:
df.head(20)

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2021-12-31 23:00:00,2022-01-01 00:00:00,10YIT-GRTN-----B,MAW,B01,677
1,2022-01-01 00:00:00,2022-01-01 01:00:00,10YIT-GRTN-----B,MAW,B01,683
2,2022-01-01 01:00:00,2022-01-01 02:00:00,10YIT-GRTN-----B,MAW,B01,670
3,2022-01-01 02:00:00,2022-01-01 03:00:00,10YIT-GRTN-----B,MAW,B01,656
4,2022-01-01 03:00:00,2022-01-01 04:00:00,10YIT-GRTN-----B,MAW,B01,649
5,2022-01-01 04:00:00,2022-01-01 05:00:00,10YIT-GRTN-----B,MAW,B01,644
6,2022-01-01 05:00:00,2022-01-01 06:00:00,10YIT-GRTN-----B,MAW,B01,642
7,2022-01-01 06:00:00,2022-01-01 07:00:00,10YIT-GRTN-----B,MAW,B01,644
8,2022-01-01 07:00:00,2022-01-01 08:00:00,10YIT-GRTN-----B,MAW,B01,649
9,2022-01-01 08:00:00,2022-01-01 09:00:00,10YIT-GRTN-----B,MAW,B01,647


In [380]:
# df = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/load_HU.csv')
print(df.shape)
# drop all the rows where green energy quantity wasn't recorded (drop the missing values)
df.drop(df[df['quantity'] == 0].index, axis=0, inplace=True)
print(df.shape)
# transform time values from string into datetime object 
df['StartTime'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['StartTime'] = pd.to_datetime(df['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

df['EndTime'] = df['EndTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['EndTime'] = pd.to_datetime(df['EndTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

"""
# Add extra columns for year, month, day, hour
df['hour'] = df['StartTime'].apply(lambda x: x.hour)
df['day'] = df['StartTime'].apply(lambda x: x.day)
df['month'] = df['StartTime'].apply(lambda x: x.month)
df['year'] = df['StartTime'].apply(lambda x: x.year)

# Remove all the samples that are missing 4 rows of 15-minute marks/records
hourly_counts = df.groupby(['year', 'month', 'day', 'hour']).size()
missing_hours = hourly_counts[hourly_counts != 4].index
df = df[~df[['year', 'month', 'day', 'hour']].apply(tuple, axis=1).isin(missing_hours)]
df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)


print(df.head(12))
# Transform data from 15-minute interval format into 1-hour interval format per sample
df = df.resample('H', on='StartTime', closed='left').sum(numeric_only=True).reset_index()
"""

(96, 6)
(96, 6)


"\n# Add extra columns for year, month, day, hour\ndf['hour'] = df['StartTime'].apply(lambda x: x.hour)\ndf['day'] = df['StartTime'].apply(lambda x: x.day)\ndf['month'] = df['StartTime'].apply(lambda x: x.month)\ndf['year'] = df['StartTime'].apply(lambda x: x.year)\n\n# Remove all the samples that are missing 4 rows of 15-minute marks/records\nhourly_counts = df.groupby(['year', 'month', 'day', 'hour']).size()\nmissing_hours = hourly_counts[hourly_counts != 4].index\ndf = df[~df[['year', 'month', 'day', 'hour']].apply(tuple, axis=1).isin(missing_hours)]\ndf.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)\n\n\nprint(df.head(12))\n# Transform data from 15-minute interval format into 1-hour interval format per sample\ndf = df.resample('H', on='StartTime', closed='left').sum(numeric_only=True).reset_index()\n"

In [18]:
df = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/IT/gen_IT_B01.csv')

# drop all the rows where green energy quantity wasn't recorded (drop the missing values)
df.drop(df[df['quantity'] == 0].index, axis=0, inplace=True)

# transform time values from string into datetime object 
df['StartTime'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['StartTime'] = pd.to_datetime(df['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))
    
df['EndTime'] = df['EndTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['EndTime'] = pd.to_datetime(df['EndTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

# Add extra columns for year, month, day, hour
df['hour'] = df['StartTime'].apply(lambda x: x.hour)
df['day'] = df['StartTime'].apply(lambda x: x.day)
df['month'] = df['StartTime'].apply(lambda x: x.month)
df['year'] = df['StartTime'].apply(lambda x: x.year)

# TODO: Before removing samples, check if the data even has 15-minute mark format
short_interval = False

time_diff = df['StartTime'].diff()

unique_diffs = time_diff.unique()

# Anything above 1-hour wouldn't matter, everything below 1 hour matters for transforming data later on into a 1-hour records
max_time_difference='01:00:00'
max_duration = pd.to_timedelta(max_time_difference)

extracted_diffs_minutes = [diff.total_seconds() / 60 for diff in unique_diffs]
filtered_list = [value for value in extracted_diffs_minutes if not np.isnan(value)]

count_req = 4
if len(filtered_list) != 1:
    print("Warn: Data's time intervals are not equal for all the samples!")
    raise Exception("TODO: Dynamically work with data which has different intervals per sample")
else:
    count_req = 60.0 / filtered_list[0]
    print("Count Req: ", count_req)

if any(diff < max_duration for diff in unique_diffs):
    print(f"Data is recorded in chunks which are shorter than {max_time_difference}h.")
    short_interval = True
else:
    print(f"Data is recorded every {max_time_difference}h or in bigger interval.")


df.drop(['EndTime', 'AreaID', 'UnitName', 'PsrType'], axis=1, inplace=True)
# Remove all the samples that are missing 4 rows of 15-minute marks/records
if short_interval:
    print("Entered")
    hourly_counts = df.groupby(['year', 'month', 'day', 'hour']).size()

    missing_hours = hourly_counts[hourly_counts != 4].index
    df = df[~df[['year', 'month', 'day', 'hour']].apply(tuple, axis=1).isin(missing_hours)]
    # Transform data from 15-minute interval format into 1-hour interval format per sample
    df = df.resample('H', on='StartTime', closed='left').sum(numeric_only=True).reset_index()

df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)

Count Req:  1.0
Data is recorded every 01:00:00h or in bigger interval.


In [19]:
df.head(20)

,StartTime,quantity,hour,day,month,year
0,2021-12-31 23:00:00,677,23,31,12,2021
1,2022-01-01 00:00:00,683,0,1,1,2022
2,2022-01-01 01:00:00,670,1,1,1,2022
3,2022-01-01 02:00:00,656,2,1,1,2022
4,2022-01-01 03:00:00,649,3,1,1,2022
5,2022-01-01 04:00:00,644,4,1,1,2022
6,2022-01-01 05:00:00,642,5,1,1,2022
7,2022-01-01 06:00:00,644,6,1,1,2022
8,2022-01-01 07:00:00,649,7,1,1,2022
9,2022-01-01 08:00:00,647,8,1,1,2022


In [214]:
#-----------------------------------------------------------
from datetime import datetime

# Example datetime string
datetime_string = "2021-12-31T23:45+00:00Z"

# Remove the 'Z' at the end and parse the string without the colon in the offset
datetime_object = datetime.strptime(datetime_string[:-1], "%Y-%m-%dT%H:%M%z")

print(datetime_object.strftime('%Y-%m-%d %H:%M'))
#-----------------------------------------------------------

2021-12-31 23:45


In [127]:
df['StartTime_DT'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['EndTime_DT'] = df['EndTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))

df2['StartTime_DT'] = df2['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df2['EndTime_DT'] = df2['EndTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))

In [300]:
df['Start'] = pd.to_datetime(df['StartTime_DT'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))
df['End'] = pd.to_datetime(df['EndTime_DT'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

df2['Start'] = pd.to_datetime(df2['StartTime_DT'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))
df2['End'] = pd.to_datetime(df2['EndTime_DT'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

# df['Start_Time'] = df['StartTime_DT'].apply(lambda x: x.time())
# df['End_Time'] = df['EndTime_DT'].apply(lambda x: x.time())

In [301]:
df.drop(['StartTime_DT', 'EndTime_DT', 'StartTime', 'EndTime'], axis=1, inplace=True)
df2.drop(['StartTime_DT', 'EndTime_DT', 'StartTime', 'EndTime'], axis=1, inplace=True)

In [218]:
df.head()

,AreaID,UnitName,PsrType,quantity,Start,End
0,10YHU-MAVIR----U,MAW,B01,132,2021-12-31 23:45:00,2022-01-01 00:00:00
1,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:00:00,2022-01-01 00:15:00
2,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:15:00,2022-01-01 00:30:00
3,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:30:00,2022-01-01 00:45:00
4,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 00:45:00,2022-01-01 01:00:00


In [194]:
df2.head()

,AreaID,UnitName,PsrType,quantity,Start,End
0,10YHU-MAVIR----U,MAW,B11,11,2021-12-31 23:45:00,2022-01-01 00:00:00
1,10YHU-MAVIR----U,MAW,B11,11,2022-01-01 00:00:00,2022-01-01 00:15:00
2,10YHU-MAVIR----U,MAW,B11,11,2022-01-01 00:15:00,2022-01-01 00:30:00
3,10YHU-MAVIR----U,MAW,B11,11,2022-01-01 00:30:00,2022-01-01 00:45:00
4,10YHU-MAVIR----U,MAW,B11,11,2022-01-01 00:45:00,2022-01-01 01:00:00


In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   AreaID    384 non-null    object        
 1   UnitName  384 non-null    object        
 2   PsrType   384 non-null    object        
 3   quantity  384 non-null    int64         
 4   Start     384 non-null    datetime64[ns]
 5   End       384 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 18.1+ KB


In [196]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   AreaID    384 non-null    object        
 1   UnitName  384 non-null    object        
 2   PsrType   384 non-null    object        
 3   quantity  384 non-null    int64         
 4   Start     384 non-null    datetime64[ns]
 5   End       384 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 18.1+ KB


In [198]:
print("DF: ", list(df.columns))
print("DF2: ", list(df2.columns))

DF:  ['AreaID', 'UnitName', 'PsrType', 'quantity', 'Start', 'End']
DF2:  ['AreaID', 'UnitName', 'PsrType', 'quantity', 'Start', 'End']


In [297]:
import datetime as dt
df.drop(df[df['Start'].dt.year < 2022].index, axis=0, inplace=True)

In [246]:
df[(df['Start'].dt.year < 2022) | (df['Start'].dt.year > 2022)]

,AreaID,UnitName,PsrType,quantity,Start,End


In [302]:
df.shape

(384, 6)

#### PRE-PROCESSING

##### TRANSFORMING INTO HOURLY INTERVAL INSTEAD OF 15-MINUTE INTERVAL (WORKS)

In [268]:
list(df.columns)

['AreaID', 'UnitName', 'PsrType', 'quantity', 'Start', 'End']

In [283]:
att1 = df.copy(deep=True)

In [284]:
att1.shape

(384, 6)

In [285]:
att1.drop(['UnitName', 'PsrType', 'Start', 'AreaID'], inplace=True, axis=1)

In [286]:
att1 = att1.resample('H', on='End', closed='right').sum().reset_index()

In [287]:
att1.head()

,End,quantity
0,2021-12-31 23:00:00,132
1,2022-01-01 00:00:00,523
2,2022-01-01 01:00:00,516
3,2022-01-01 02:00:00,507
4,2022-01-01 03:00:00,506


In [261]:
df.head()

,AreaID,UnitName,PsrType,quantity,Start,End
1,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:00:00,2022-01-01 00:15:00
2,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:15:00,2022-01-01 00:30:00
3,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:30:00,2022-01-01 00:45:00
4,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 00:45:00,2022-01-01 01:00:00
5,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 01:00:00,2022-01-01 01:15:00


In [303]:
att2 = df.copy(deep=True)

In [304]:
att2.head()

,AreaID,UnitName,PsrType,quantity,Start,End
0,10YHU-MAVIR----U,MAW,B01,132,2021-12-31 23:45:00,2022-01-01 00:00:00
1,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:00:00,2022-01-01 00:15:00
2,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:15:00,2022-01-01 00:30:00
3,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:30:00,2022-01-01 00:45:00
4,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 00:45:00,2022-01-01 01:00:00


In [316]:
att2['hour'] = att2['Start'].apply(lambda x: x.hour)
att2['day'] = att2['Start'].apply(lambda x: x.day)
att2['month'] = att2['Start'].apply(lambda x: x.month)
att2['year'] = att2['Start'].apply(lambda x: x.year)

In [328]:
hourly_counts = att2.groupby(['year', 'month', 'day', 'hour']).size()

# Filter out hours with less than 4 intervals
missing_hours = hourly_counts[hourly_counts != 4].index

In [329]:
missing_hours

MultiIndex([(2021, 12, 31, 23),
            (2022,  1,  4, 23)],
           names=['year', 'month', 'day', 'hour'])

In [330]:
# df_filtered = att2[~att2['hour'].isin(missing_hours)]
df_filtered = att2[~att2[['year', 'month', 'day', 'hour']].apply(tuple, axis=1).isin(missing_hours)]

In [331]:
df_filtered.head()

,AreaID,UnitName,PsrType,quantity,Start,End,hour,month,day,year
1,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:00:00,2022-01-01 00:15:00,0,1,1,2022
2,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:15:00,2022-01-01 00:30:00,0,1,1,2022
3,10YHU-MAVIR----U,MAW,B01,131,2022-01-01 00:30:00,2022-01-01 00:45:00,0,1,1,2022
4,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 00:45:00,2022-01-01 01:00:00,0,1,1,2022
5,10YHU-MAVIR----U,MAW,B01,130,2022-01-01 01:00:00,2022-01-01 01:15:00,1,1,1,2022


#### CONCAT/MERGE ATTEMPT

In [199]:
test_df = pd.merge(
    df,
    df2,
    how='inner',
    on='Start',
    copy=True
)

In [211]:
df.tail()

,AreaID,UnitName,PsrType,quantity,Start,End
379,10YHU-MAVIR----U,MAW,B01,135,2022-01-04 22:30:00,2022-01-04 22:45:00
380,10YHU-MAVIR----U,MAW,B01,130,2022-01-04 22:45:00,2022-01-04 23:00:00
381,10YHU-MAVIR----U,MAW,B01,129,2022-01-04 23:00:00,2022-01-04 23:15:00
382,10YHU-MAVIR----U,MAW,B01,129,2022-01-04 23:15:00,2022-01-04 23:30:00
383,10YHU-MAVIR----U,MAW,B01,128,2022-01-04 23:30:00,2022-01-04 23:45:00


In [212]:
test_df.tail()

,AreaID_x,UnitName_x,PsrType_x,quantity_x,Start,End_x,AreaID_y,UnitName_y,PsrType_y,quantity_y,End_y
379,10YHU-MAVIR----U,MAW,B01,135,2022-01-04 22:30:00,2022-01-04 22:45:00,10YHU-MAVIR----U,MAW,B11,6,2022-01-04 22:45:00
380,10YHU-MAVIR----U,MAW,B01,130,2022-01-04 22:45:00,2022-01-04 23:00:00,10YHU-MAVIR----U,MAW,B11,6,2022-01-04 23:00:00
381,10YHU-MAVIR----U,MAW,B01,129,2022-01-04 23:00:00,2022-01-04 23:15:00,10YHU-MAVIR----U,MAW,B11,6,2022-01-04 23:15:00
382,10YHU-MAVIR----U,MAW,B01,129,2022-01-04 23:15:00,2022-01-04 23:30:00,10YHU-MAVIR----U,MAW,B11,6,2022-01-04 23:30:00
383,10YHU-MAVIR----U,MAW,B01,128,2022-01-04 23:30:00,2022-01-04 23:45:00,10YHU-MAVIR----U,MAW,B11,6,2022-01-04 23:45:00


In [178]:
df.loc[:, ['PsrType', 'quantity', 'Start']].groupby(['Start', 'PsrType']).mean()

,,quantity
Start,PsrType,
2021-12-31 23:45:00,B01,132.0
2022-01-01 00:00:00,B01,131.0
2022-01-01 00:15:00,B01,131.0
2022-01-01 00:30:00,B01,131.0
2022-01-01 00:45:00,B01,130.0
...,...,...
2022-01-04 22:30:00,B01,135.0
2022-01-04 22:45:00,B01,130.0
2022-01-04 23:00:00,B01,129.0


#### MERGE ALL

In [231]:
df['StartTime'].min()

Timestamp('2022-01-01 00:00:00')

In [235]:
tstmp = pd.Timestamp(start_time)

In [238]:
tstmp.replace(minute=30)

Timestamp('2022-01-01 00:30:00')

In [253]:
# TODO:
#   - Change string object of datetime into a datetime object - [DONE]
#   - Add columns for year, month, day, hour for groupby method - 
#   - Remove all the rows which don't have 4 consecutive samples of 15-minute interval scans (usually first and last samples in data)
#   - Transform data so each sample represents a 1-hour interval with summed up quantity values
#   - Rinse and repeat for all files
#   - Merge and sum all the green energy quantities into a single column value, extract .csv data into subdirectory it belongs to

def load_dataframe(path):
    df = pd.read_csv(path)

    return df

def fill_missing_hours(df, frequency, df_type):
    end_time=pd.Timestamp(datetime(2022, 1, 5))
    start_time=pd.Timestamp(datetime(2022, 1, 1))
    
    min_start_time = df['StartTime'].min()
    max_start_time = df['StartTime'].max()

    print("MIN AND MAX BEFORE:")
    print(min_start_time)
    print(max_start_time)
    print('-' * 3)

    if min_start_time > start_time:
        min_start_time = start_time.replace(minute=0)
    else:
        min_start_time = min_start_time.replace(minute=0)
    
    if max_start_time < end_time:
        max_start_time = end_time.replace(minute=0)
    else:
        max_start_time = max_start_time.replace(minutes=0)
    
    print("MIN AND MAX AFTER")
    print(min_start_time)
    print(max_start_time)

    complete_range = pd.date_range(start=min_start_time, end=max_start_time, freq=f'{int(frequency)}T')
    complete_df = pd.DataFrame({'StartTime': complete_range})

    # Merge the original dataframe with the complete dataframe
    merged_df = pd.merge(complete_df, df, on='StartTime', how='left')

    # Fill missing values with 0 or any desired value
    merged_df[df_type] = merged_df[df_type].fillna(0)

    # Print or use merged_df as needed
    # print(merged_df.head())
    return merged_df

def pre_process(file_name, dir_path):
    print(f"PATH: {dir_path}/{file_name}")
    df = load_dataframe(dir_path + file_name)

    df.drop_duplicates('StartTime', keep='first', inplace=True)    
    df.drop(['EndTime', 'AreaID', 'UnitName'], axis=1, inplace=True)
    
    # transform time values from string into datetime object 
    df['StartTime'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
    df['StartTime'] = pd.to_datetime(df['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

    # TODO: Before removing samples, check if the data even has 15-minute mark format
    short_interval = False

    time_diff = df['StartTime'].diff()

    unique_diffs = time_diff.unique()

    # Anything above 1-hour wouldn't matter, everything below 1 hour matters for transforming data later on into 1-hour records
    max_time_difference='01:00:00'
    max_duration = pd.to_timedelta(max_time_difference)

    extracted_diffs_minutes = [diff.total_seconds() / 60 for diff in unique_diffs]
    filtered_list = [value for value in extracted_diffs_minutes if not np.isnan(value) and value > 0.0]
    # filtered_list = [value for value in filtered_list if value > 0.0]

    count_req = 4

    if len(filtered_list) != 1 and min(filtered_list) < 60.0:
        # print("Warn: Data's time intervals are not equal for all the samples!")
        # print("FILTERED LIST: ", filtered_list)
        count_req = np.floor(60.0 / min(filtered_list))
        # raise Exception("TODO: Dynamically work with data which has different intervals per sample")

    if min(filtered_list) < 60.0:
        # print("SHORT_INTERVAL=TRUE, MIN_INTERVAL = ", min(filtered_list))
        short_interval = True

    # if len(filtered_list) != 1:
        # print("Entered here:")
        # print("Frequency: ", min(filtered_list))
    if min(filtered_list) > 60.0:
        freq = 60.0
    else:
        freq = min(filtered_list)
        
        print("-" * 10)
        print(df.head(5))
        print('-' * 10)
    if 'quantity' in list(df.columns):
        df = fill_missing_hours(df, frequency=freq, df_type='quantity')
    else:
        df = fill_missing_hours(df, frequency=freq, df_type='Load')
    
    # Interpolate missing values and check if there's any data within dataframe we can use (if all values are 0 in the entire dataframe, we have no use of it)
    if 'PsrType' in list(df.columns):
        if df.loc[df['quantity'] == 0].shape[0] == df.shape[0]:
            return None
        df.drop(['PsrType'], axis=1, inplace=True)
        df['quantity'] = df['quantity'].replace(0, np.nan)
        df.interpolate(method='linear', limit_direction='both', inplace=True)
        #df.drop(df[df['quantity'] == 0].index, axis=0, inplace=True)
    else:
        if df.loc[df['Load'] == 0].shape[0] == df.shape[0]:
            return None
        df['Load'].replace(0, np.nan, inplace=True)
        df['Load'].interpolate(method='linear', limit_direction='both', inplace=True)
        #df.drop(df[df['Load'] == 0].index, axis=0, inplace=True)

    # Add extra columns for year, month, day, hour
    df['hour'] = df['StartTime'].apply(lambda x: x.hour)
    df['day'] = df['StartTime'].apply(lambda x: x.day)
    df['month'] = df['StartTime'].apply(lambda x: x.month)
    df['year'] = df['StartTime'].apply(lambda x: x.year)

    print("BEFORE:")
    # print(df.tail())
    # Remove all the samples that are missing 4 rows of 15-minute marks/records
    if short_interval:
        # print("COUNT REQ: ", count_req)
        hourly_counts = df.groupby(['year', 'month', 'day', 'hour']).size()

        missing_hours = hourly_counts[hourly_counts != count_req].index
        print(missing_hours)

        df = df[~df[['year', 'month', 'day', 'hour']].apply(tuple, axis=1).isin(missing_hours)]
        
        # Transform data from 15-minute interval format into 1-hour interval format per sample
        df = df.resample('H', on='StartTime', closed='left').sum(numeric_only=True).reset_index()
    
    df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)

    return df


def merge_gen_dataframes(dfs, region_code):
    """
    merged_df = pd.merge(df_1, df_2, on=['StartTime', 'quantity'], how='outer')

    result_df = merged_df.groupby('StartTime')['quantity'].sum().reset_index()
    """
    # print('-' * 10)
    # for df in dfs:
    #     print(df.head(10))
    #     print('-' * 5)
        
    # print('-' * 10)
    def custom_merge(df_left, df_right):
        merged_df = pd.merge(df_left, df_right, on=['StartTime'], how='outer', suffixes=('', '_other'))
        # Sum up the 'quantity' values from both dataframes
        merged_df['quantity'] = merged_df['quantity'] + merged_df['quantity_other']
        # Drop the redundant 'quantity_other' column
        merged_df = merged_df.drop('quantity_other', axis=1)
        return merged_df

    df_merged = reduce(custom_merge, dfs)

    # df_merged = reduce(lambda left, right: pd.merge(left, right, on=['StartTime', 'quantity'], how='outer'), dfs)
    # df_merged = df_merged.groupby('StartTime')['quantity'].sum().reset_index()
    df_merged.rename(columns={'quantity': f'green_energy_{region_code}'}, inplace=True)
    
    # print(df_merged.head())
    # print()
    # print(list(df_merged.columns))
    return df_merged

In [208]:
df = pd.read_csv('/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/gen_HU_B01.csv')

In [254]:
df = pre_process('gen_HU_B01.csv', '/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/')

PATH: /home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU//gen_HU_B01.csv
----------
            StartTime PsrType  quantity
0 2021-12-31 23:45:00     B01       132
1 2022-01-01 00:00:00     B01       131
2 2022-01-01 00:15:00     B01       131
3 2022-01-01 00:30:00     B01       131
4 2022-01-01 00:45:00     B01       130
----------
MIN AND MAX BEFORE:
2021-12-31 23:45:00
2022-01-04 23:30:00
---
MIN AND MAX AFTER
2021-12-31 23:00:00
2022-01-05 00:00:00
BEFORE:
MultiIndex([(2022, 1, 5, 0)],
           names=['year', 'month', 'day', 'hour'])


In [256]:
df.tail()

,StartTime,quantity
92,2022-01-04 19:00:00,537.0
93,2022-01-04 20:00:00,539.0
94,2022-01-04 21:00:00,542.0
95,2022-01-04 22:00:00,538.0
96,2022-01-04 23:00:00,514.0


In [205]:
df['StartTime'] = df['StartTime'].apply(lambda x: datetime.strptime(x[:-1], "%Y-%m-%dT%H:%M%z"))
df['StartTime'] = pd.to_datetime(df['StartTime'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

In [206]:
df = fill_missing_hours(df, frequency=30.0, df_type='quantity')

            StartTime                  EndTime            AreaID UnitName  \
0 2021-12-31 23:45:00  2022-01-01T00:00+00:00Z  10YHU-MAVIR----U      MAW   
1 2022-01-01 00:15:00  2022-01-01T00:30+00:00Z  10YHU-MAVIR----U      MAW   
2 2022-01-01 00:45:00  2022-01-01T01:00+00:00Z  10YHU-MAVIR----U      MAW   
3 2022-01-01 01:15:00  2022-01-01T01:30+00:00Z  10YHU-MAVIR----U      MAW   
4 2022-01-01 01:45:00  2022-01-01T02:00+00:00Z  10YHU-MAVIR----U      MAW   

  PsrType  quantity  
0     B01       132  
1     B01       131  
2     B01       130  
3     B01       129  
4     B01       128  


In [207]:
df.tail()

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
187,2022-01-04 21:15:00,2022-01-04T21:30+00:00Z,10YHU-MAVIR----U,MAW,B01,136
188,2022-01-04 21:45:00,2022-01-04T22:00+00:00Z,10YHU-MAVIR----U,MAW,B01,136
189,2022-01-04 22:15:00,2022-01-04T22:30+00:00Z,10YHU-MAVIR----U,MAW,B01,137
190,2022-01-04 22:45:00,2022-01-04T23:00+00:00Z,10YHU-MAVIR----U,MAW,B01,130
191,2022-01-04 23:15:00,2022-01-04T23:30+00:00Z,10YHU-MAVIR----U,MAW,B01,129


In [51]:
dir_path = "/home/stefan/Documents/Workspace/Python/ML/Hackathon/eco-forecast/data/HU/"
file_name = "gen_HU_B01.csv"
df_1 = pre_process(file_name, dir_path)
file_name = "gen_HU_B02.csv"
df_2 = pre_process(file_name, dir_path)

DF SHAPE:  (384, 5)
FILTERED LIST:  [15.0]
Count Req:  4.0
Data is recorded in chunks which are shorter than 01:00:00h.
DF SHAPE:  (384, 5)
FILTERED LIST:  [15.0]
Count Req:  4.0
Data is recorded in chunks which are shorter than 01:00:00h.


In [56]:
df_1.head()

,StartTime,quantity
0,2022-01-01 00:00:00,523
1,2022-01-01 01:00:00,516
2,2022-01-01 02:00:00,507
3,2022-01-01 03:00:00,506
4,2022-01-01 04:00:00,504


In [57]:
df_2.head()

,StartTime,quantity
0,2022-01-01 00:00:00,1225
1,2022-01-01 01:00:00,1233
2,2022-01-01 02:00:00,1174
3,2022-01-01 03:00:00,1178
4,2022-01-01 04:00:00,1219


In [54]:
merged_df = pd.merge(df_1, df_2, on=['StartTime', 'quantity'], how='outer')

result_df = merged_df.groupby('StartTime')['quantity'].sum().reset_index()

In [55]:
result_df.head()

,StartTime,quantity
0,2022-01-01 00:00:00,1748
1,2022-01-01 01:00:00,1749
2,2022-01-01 02:00:00,1681
3,2022-01-01 03:00:00,1684
4,2022-01-01 04:00:00,1723


In [198]:
# All that's left is to merge each of the dataframes saved within df_greens into a singular dataframe where quantities are summed up
# Afterwards merge the df_load with the above mentioned dataframe, change column names so they're called Green_XX and Load_XX where XX is the region code (dir value)
df_merges = []
df_merged = None
for dir in data_dirs:
    # dataframe name (dir = region code)
    # df_name = f'df_{dir}'

    dir_path = f'{DATA_PATH}/{dir}/'
    files = os.listdir(dir_path)
    # print("FILES: ", files)

    df_greens = []
    df_load = None
    for file_name in files:   
        if 'merged' in file_name.split('_'):
            continue
        if 'load' not in file_name.split('_'):           
            # extract ptr type from the file name
            # print(file_name)
            ptr_type = file_name.split('_')[2].split('.')[0]
            
            # check if the extracted ptr type is considered green energy
            if ptr_type in list(green_energy_sources.keys()):
                # print("FILE: ", file_name)
                df = pre_process(file_name, dir_path)

                if df is None:
                    print("DF IS NONE")
                    continue
                df_greens.append(df)
        else:
            df_load = pre_process(file_name, dir_path)
            df_load.rename(columns={'Load':f'{dir}_Load'}, inplace=True)
    
    # MERGE ALL FROM DF_GREENS INTO 1 AND SAVE IN ITS SUBDIRECTORY FOLDER OR SAVE IN ANOTHER LIST?
    if len(df_greens) != 0:
        df_merged = merge_gen_dataframes(df_greens, region_code=dir)
        df_merged = pd.merge(df_merged, df_load, how='outer')
    else:
        df_merged = df_load

    df_merged.to_csv(f'{DATA_PATH}/{dir}/merged_{dir}.csv')
    df_merges.append(df_merged)

raw_data = reduce(lambda left, right: pd.merge(left, right, on=['StartTime'], how='outer'), df_merges)
if raw_data.isna().sum().sum() > 0:
    raw_data.interpolate('linear', limit_direction='both', inplace=True)

raw_data.to_csv(f'{DATA_PATH}/raw_data.csv')
# df_merged.rename(columns={'quantity': f'green_energy_{region_code}'}, inplace=True)


PATH: ../data//SE//load_SE.csv
PATH: ../data//SE//gen_SE_B16.csv
PATH: ../data//SE//gen_SE_B12.csv
PATH: ../data//SE//gen_SE_B19.csv
PATH: ../data//SE//gen_SE_B13.csv
DF IS NONE
PATH: ../data//DE//gen_DE_B19.csv
PATH: ../data//DE//gen_DE_B16.csv
PATH: ../data//DE//gen_DE_B12.csv
PATH: ../data//DE//load_DE.csv
PATH: ../data//DE//gen_DE_B17.csv
PATH: ../data//DE//gen_DE_B15.csv
PATH: ../data//DE//gen_DE_B09.csv
PATH: ../data//DE//gen_DE_B01.csv
PATH: ../data//DE//gen_DE_B11.csv
PATH: ../data//DE//gen_DE_B18.csv
PATH: ../data//DE//gen_DE_B10.csv
PATH: ../data//HU//gen_HU_B19.csv
PATH: ../data//HU//gen_HU_B12.csv
PATH: ../data//HU//gen_HU_B16.csv
PATH: ../data//HU//gen_HU_B11.csv
PATH: ../data//HU//gen_HU_B17.csv
PATH: ../data//HU//gen_HU_B01.csv
PATH: ../data//HU//gen_HU_B09.csv
PATH: ../data//HU//load_HU.csv
PATH: ../data//HU//gen_HU_B15.csv
PATH: ../data//IT//gen_IT_B11.csv
PATH: ../data//IT//gen_IT_B19.csv
PATH: ../data//IT//gen_IT_B12.csv
PATH: ../data//IT//load_IT.csv
PATH: ../data//

In [203]:
b = None

if b is not None:
    print("a")

In [160]:
if raw_data.isna().sum().sum() > 0:
    raw_data.interpolate('linear', limit_direction='both', inplace=True)

In [199]:
raw_data.tail()

,StartTime,green_energy_SE,SE_Load,green_energy_DE,DE_Load,green_energy_HU,HU_Load,green_energy_IT,IT_Load,green_energy_SP,SP_Load,green_energy_NE,NE_Load,green_energy_PO,PO_Load,UK_Load,green_energy_DK,DK_Load
91,2022-01-04 18:00:00,13325.0,20499,122721.0,270170.0,1698.0,23676.0,9200.0,35807,20687.0,32753,12375.0,59419.0,3583.0,23112,2790.0,5540,4855
92,2022-01-04 19:00:00,13043.0,19641,122615.0,254412.0,1684.0,22830.0,8795.0,33021,21204.0,32032,12435.0,56378.0,3672.0,21669,2566.0,5721,4594
93,2022-01-04 20:00:00,13089.0,19007,118980.0,242520.0,1461.0,21785.0,8561.0,29495,20950.0,29607,13827.0,53698.0,3580.0,20229,2383.0,5703,4322
94,2022-01-04 21:00:00,12775.0,18021,118749.0,230236.0,1557.0,20845.0,8357.0,26393,19886.0,26964,15304.0,51094.0,3744.0,18802,2180.0,5612,4036
95,2022-01-04 22:00:00,12698.0,17745,119999.0,216521.0,1545.0,19721.0,8238.0,23875,19101.0,25526,15268.0,48171.0,3889.0,17752,1964.0,5652,3905


In [68]:
df_merged.loc[(df_merged['green_energy_SE'] == 0) | (df_merged['SE_Load'] == 0)]

,StartTime,green_energy_SE,SE_Load


In [226]:
import datetime
end_time=datetime.datetime(2022, 1, 5)
start_time=datetime.datetime(2022, 1, 1)

In [233]:
pd.Timestamp(start_time)

Timestamp('2022-01-01 00:00:00')